<h1 align='center'><b>Фреймворк PyTorch для разработки искусственных нейронных сетей<b></h1>

<h2 align='center'>Урок 9. Трансформер</h2>

<h3 align='left'>Практическое задание:</h3>  

Домашнее задание
Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
Сделайте предсказания на всем df_val. Посчитайте метрику качества.
Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.



## 0. Подготовка
Подключаем нужные модули и библиотеки

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 5.9 MB/s eta 0:00:00


In [2]:
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel, pipeline

Определим нужное нам устройство:

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device_int = 0 if torch.cuda.is_available() else -1
device

'cuda'

## 2. Загрузка данных

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2023-02-25 03:51:35--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 142.250.4.100, 142.250.4.101, 142.250.4.138, ...
Connecting to drive.google.com (drive.google.com)|142.250.4.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/96tc869uuo6r15lu1m96q91lfj99uihr/1677297075000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=f37e0872-c284-4d9c-ba52-6ff83e0fa8c0 [following]
--2023-02-25 03:51:39--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/96tc869uuo6r15lu1m96q91lfj99uihr/1677297075000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=f37e0872-c284-4d9c-ba52-6ff83e0fa8c0
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 142.250.4.132, 2404:6800:

In [6]:
!unzip data.zip

Archive:  data.zip
  inflating: train.csv               
  inflating: val.csv                 


In [7]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_train.drop('id', axis=1, inplace=True)
df_val.drop('id', axis=1, inplace=True)
df_train.shape, df_val.shape

((181467, 2), (22683, 2))

In [8]:
df_train.head()

,text,class
0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [9]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [10]:
df_val['class'].value_counts()

1    11449
0    11234
Name: class, dtype: int64

In [11]:
idx = 16
print(df_train.iloc[idx]['text'])

RT @SimonenkoYulia: @anichkabudnik Я ТОЖИ ЛАХ НО ЗАВТРА НЕ ИДУ)


In [12]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())
print(df_train.iloc[idx]['text'])

rt @simonenkoyulia: @anichkabudnik я тожи лах но завтра не иду)


- input_ids - закодированные айди токенов
- attention_mask нужна для того, чтобы PAD символы не проходили через трансформер

## 1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.

In [13]:
tokenizer = AutoTokenizer.from_pretrained("Tatyana/rubert-base-cased-sentiment-new")
base_model = AutoModel.from_pretrained("Tatyana/rubert-base-cased-sentiment-new")

sentiment = pipeline("text-classification", model='Tatyana/rubert-base-cased-sentiment-new', return_all_scores=True, device=device_int)
sentiment("Обычный день")

Some weights of the model checkpoint at Tatyana/rubert-base-cased-sentiment-new were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEUTRAL', 'score': 0.49175795912742615},
  {'label': 'POSITIVE', 'score': 0.21006068587303162},
  {'label': 'NEGATIVE', 'score': 0.298181414604187}]]

In [14]:
base_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

## 2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.

In [15]:
tokens = tokenizer(df_val['text'].tolist())['input_ids']
tokens[0]

[101,
 88488,
 168,
 270,
 10382,
 10874,
 12181,
 11110,
 4416,
 1330,
 156,
 37983,
 48512,
 106,
 122,
 122,
 122,
 110,
 545,
 1790,
 60556,
 47829,
 156,
 134,
 134,
 270,
 132,
 11682,
 134,
 276,
 29395,
 265,
 137,
 279,
 245,
 275,
 10267,
 102]

In [16]:
max_len = max([len(i) for i in tokens])
max_len

110

In [17]:
mean_len = sum([len(i) for i in tokens]) / len(tokens)
mean_len

27.15663712912754

In [27]:
MAX_LENGTH = 68

In [28]:
%%time

df_val['pred'] = sentiment(df_val['text'].tolist(), padding='max_length', max_length=MAX_LENGTH, truncation=True)

CPU times: user 4min 19s, sys: 740 ms, total: 4min 20s
Wall time: 4min 27s


In [29]:
df_val['pred'][0]

[{'label': 'NEUTRAL', 'score': 0.0059600514359772205},
 {'label': 'POSITIVE', 'score': 0.990689754486084},
 {'label': 'NEGATIVE', 'score': 0.0033502262085676193}]

In [30]:
df_val['pred'] = df_val['pred'].apply(lambda res: int(res[1]['score'] > res[2]['score']))
df_val['pred'].head()

0    1
1    0
2    0
3    0
4    0
Name: pred, dtype: int64

In [31]:
df_val['pred'].value_counts()

1    11394
0    11289
Name: pred, dtype: int64

In [32]:
val_acc = (df_val['class'] == df_val['pred']).sum() / len(df_val['class'])
val_acc

0.9963408720186924

In [33]:
errors = df_val[df_val['class'] != df_val['pred']]
errors.head()

,text,class,pred
569,что в этих ваших дебианах используют для вебка...,1,0
642,"@echelon_2001 спасибо ^_^ но все-равно, как-то...",1,0
763,"rt @fnscjuravl: друг рассказал, что в его горо...",1,0
992,хочу пожелать доброго утра и хорошего дня для ...,1,0
1423,^_^с этим просто не сравница^_^ http://t.co/7m...,1,0


In [34]:
print("Type 1 error (FP): ", len(errors[errors['class'] == 0]))
print("Type 2 error (FN): ", len(errors[errors['class'] == 1]))

Type 1 error (FP):  14
Type 2 error (FN):  69


## 3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

In [35]:
class Dataset(Dataset):
    
    def __init__(self, txts, labels, tokenizer, max_length):
        self.labels = torch.LongTensor(labels)
        self.txts = tokenizer(txts, padding='max_length', 
                                max_length=max_length, truncation=True)

    def __getitem__(self, index):
        return self.txts['input_ids'][index], self.txts['attention_mask'][index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)

In [36]:
train_data = Dataset(df_train['text'].tolist(), df_train['class'], tokenizer, MAX_LENGTH)
test_data = Dataset(df_val['text'].tolist(), df_val['class'], tokenizer, MAX_LENGTH)

In [37]:
train_loader = DataLoader(train_data, batch_size=512, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=256, shuffle=True, num_workers=1)

In [38]:
class RuBertClassifier(nn.Module):

    def __init__(self, base_model, hidden_dim, dropout):
        super().__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout)
        self.linear_1 = nn.Linear(768, hidden_dim)
        self.linear_2 = nn.Linear(hidden_dim, 2)

    def forward(self, x, mask):
        
        _, pooled_output = self.base_model(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_1_output = self.linear_1(dropout_output)
        tanh_output = torch.tanh(linear_1_output)
        linear_2_output = self.linear_2(tanh_output)
        sigm_output = torch.sigmoid(linear_2_output)
        return sigm_output

In [39]:
model = RuBertClassifier(base_model, hidden_dim=384, dropout=0.2).to(device)

In [40]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [41]:
all_param = sum([param.nelement() for param in model.parameters()])
train_param = all_param - sum([param.nelement() for param in model.base_model.parameters()])
print("Parameters full train:", all_param)
print("Parameters transfer learning:", train_param)

Parameters full train: 178149506
Parameters transfer learning: 296066


In [42]:
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [43]:
epochs = 2

for epoch_num in range(epochs):
    train_loss = 0
    test_loss = 0
    train_acc = 0
    test_acc = 0

    model.train()
    for input_id, mask, labels in tqdm(train_loader):
        mask = torch.stack(mask).permute(1,0).to(device)
        input_id = torch.stack(input_id).permute(1,0).to(device)
        labels = labels.to(device)

        output = model(input_id, mask)
                
        loss = criterion(output, labels)
        train_loss += loss.item()
                
        train_acc += (output.argmax(dim=1) == labels).sum().item() / len(labels)

        model.zero_grad()
        loss.backward()
        optimizer.step()
            
    model.eval()
    for input_id, mask, labels in test_loader:
        mask = torch.stack(mask).permute(1,0).to(device)
        input_id = torch.stack(input_id).permute(1,0).to(device)
        labels = labels.to(device)

        output = model(input_id, mask)

        loss = criterion(output, labels)
        test_loss += loss.item()
                    
        test_acc += (output.argmax(dim=1) == labels).sum().item() / len(labels)
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {train_loss / len(train_loader): .3f} \
        | Train Accuracy: {train_acc / len(train_loader): .6f} \
        | Test Loss: {test_loss / len(test_loader): .3f} \
        | Test Accuracy: {test_acc / len(test_loader): .6f}')

100%|██████████| 355/355 [10:45<00:00,  1.82s/it]


Epochs: 1 | Train Loss:  0.318         | Train Accuracy:  0.994669         | Test Loss:  0.317         | Test Accuracy:  0.996328


100%|██████████| 355/355 [10:45<00:00,  1.82s/it]


Epochs: 2 | Train Loss:  0.317         | Train Accuracy:  0.996402         | Test Loss:  0.317         | Test Accuracy:  0.996401


## Вывод:

Я загрузил RuBert модель из библиотеки transformers и использовал её для бинарной задачи прогнозирования эмоциональной окраски текста (sentiment analysis). 

Если брать все слова в предложениях, то есть MAX_LENGTH = 68 (среднее между средней и максимальной длиной), результат оказался достаточно хорошим, 0.9963408720186924 точность предсказания (accuracy). При этом можно заметить, что ошибок первого рода (FP) гораздо меньше ошибок второго рода (FN).  

В виде классификатора возьмём два линейных слоя, при этом будет 2 выхода с активацией sigmoid, вероятность для каждого класса.

После дообучения видно, что Train Loss падает, Train Accuracy растёт, Test Accuracy растёт, значит наша модель правильно дообучается. Так же видно, что итоговая метрика Test Accuracy с дообучением больше, чем без, значит стратегия дообучения классификатора модели для конкретных данных работает.